In [1]:
import tensorflow as tf
physicalDevices = tf.config.list_physical_devices('GPU') 
if physicalDevices:
    tf.config.experimental.set_memory_growth(physicalDevices[0], True)  
from tensorflow import keras
from tensorflow.keras import layers
from data_preprocessor import DataGenerator
from gestures import GESTURES


import datetime

In [4]:
batch_size=64
epochs=50
input_shape = (10,21,3)
classes = len(GESTURES)
trainGen = DataGenerator('gesture_dataset.h5',batchSize=batch_size, dim=input_shape)

In [5]:
inputs = layers.Input(shape=input_shape)
x = inputs
x = layers.TimeDistributed(layers.Flatten())(x)
x = layers.TimeDistributed(layers.Dropout(0.5))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.5))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.3))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.2))(x)
x = layers.GRU(64)(x)
output = layers.Dense(classes, activation='softmax',name='Output')(x)
model = keras.models.Model(inputs, output)
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10, 21, 3)]       0         
_________________________________________________________________
time_distributed (TimeDistri (None, 10, 63)            0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 63)            0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 10, 64)            4096      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 10, 64)            0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 10, 64)            4160      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 10, 64)            0     

In [6]:
model.fit(trainGen,epochs=epochs)

Epoch 1/50
45/45 [==============================] - 4s 8ms/step - loss: 1.9544 - accuracy: 0.2167
Epoch 2/50
45/45 [==============================] - 0s 7ms/step - loss: 0.9312 - accuracy: 0.7156
Epoch 3/50
45/45 [==============================] - 0s 6ms/step - loss: 0.2807 - accuracy: 0.9188
Epoch 4/50
45/45 [==============================] - 0s 10ms/step - loss: 0.1501 - accuracy: 0.9542
Epoch 5/50
45/45 [==============================] - 0s 6ms/step - loss: 0.1016 - accuracy: 0.9713
Epoch 6/50
45/45 [==============================] - 0s 6ms/step - loss: 0.0941 - accuracy: 0.9750
Epoch 7/50
45/45 [==============================] - 0s 7ms/step - loss: 0.0747 - accuracy: 0.9787
Epoch 8/50
45/45 [==============================] - 0s 7ms/step - loss: 0.0590 - accuracy: 0.9833
Epoch 9/50
45/45 [==============================] - 0s 6ms/step - loss: 0.0581 - accuracy: 0.9850
Epoch 10/50
45/45 [==============================] - 0s 7ms/step - loss: 0.0419 - accuracy: 0.9883
Epoch 11/50
45/45 

In [7]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tfliteModel = converter.convert()
modelPath = "saved_models/MODEL-{}.tflite".format(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
with open(modelPath, 'wb') as f:
  f.write(tfliteModel)

INFO:tensorflow:Assets written to: C:\Users\17ami\AppData\Local\Temp\tmp3ff81726\assets
INFO:tensorflow:Assets written to: C:\Users\17ami\AppData\Local\Temp\tmp3ff81726\assets
